<a href="https://colab.research.google.com/github/fernandodeeke/can2025/blob/main/gaussian_elimination3_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <h2></h2></h1></center>
<center><h1>Numerical Analysis</h1></center>
<center><h2>2025/1</h2></center>
<center><h3>Fernando Deeke Sasse</h3></center>
<center><h3>CCT - UDESC</h3></center>
<center><h2>Gaussian Elimination - 3</h2></center>

### 1. Partial row pivoting

Let's consider the linear system $AX = B$ with

$$
A = \begin{bmatrix}
10^{-8} & 1 & 4 \\
1 & 1 & 4 \\
2 & 4 & 5
\end{bmatrix}\,,\,\,
B = \begin{bmatrix}
1 \\ 3\\ 4
\end{bmatrix}
$$

Let's solve this system using the GaussSolve algorithm, presented earlier:

In [1]:
import numpy as np

In [2]:
def GaussSolve(a,b):
    n = len(a)
    x=np.zeros(n)
    for k in range(0,n-1):
        for i in range(k+1,n):
            lam = a[i,k]/a[k,k]
            a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
            b[i] = b[i] - lam*b[k]
    x[n-1]=b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (b[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
    return np.transpose([x])

We define the matrices:

In [6]:
A = np.array([[1.e-8, 1.,4.],[1.e+8, 1.,4.],[2.,4.,5.]])
B = np.array([1.,3.,4.])

Let us test the algorithm:

In [7]:
X1 = GaussSolve(A,B)
X1

array([[4.44089210e-08],
       [9.99999967e-01],
       [8.12790611e-09]])

We may verify the result calculating the residue:

In [12]:
A = np.array([[1.e-8, 1.,4.],[1.e+8, 1.,4.],[2.,4.,5.]])
B = np.array([1.,3.,4.])
Bt = B.reshape(3,1)

In [13]:
R1=A@X1-Bt
R1

array([[ 0.00000000e+00],
       [ 2.44089210e+00],
       [-5.89126969e-10]])

We can note that there is a problem with this solution, as the second component is not close to zero. Such a problem can be solved by an exchange of rows:

In [15]:
A2 = np.array([[1.e+8, 1.,4.],[1.e-8, 1.,4.],[2.,4.,5.]])
B2 = np.array([3.,1.,4.])
B2t = B2.reshape(3,1)

Let us try again:

In [16]:
X2 = GaussSolve(A2,B2)
X2

array([[2.00000000e-08],
       [9.99999985e-01],
       [3.63636357e-09]])

Let us calculate the residue:

In [17]:
A2 = np.array([[1.e+8, 1.,4.],[1.e-8, 1.,4.],[2.,4.,5.]])
B2 = np.array([3.,1.,4.])
B2t = B2.reshape(3,1)

In [18]:
R2=A2@X2-B2t
R2

array([[-4.4408921e-16],
       [ 0.0000000e+00],
       [ 0.0000000e+00]])

which shows that the solution is correct.

To explain the problem of the first procedure, let us examine in detail the origin of the problem in the first calculation. Let us form the enlarged matrix and perform the Gaussian elimination step by step

In [19]:
A = np.array([[1.e-8, 1.,4.],[1.e+8, 1.,4.],[1.,4.,5.]])
B = np.array([1.,3.,4.])
Bt = B.reshape(3,1)

In [20]:
M = np.hstack([A,Bt])
print(M)

[[1.e-08 1.e+00 4.e+00 1.e+00]
 [1.e+08 1.e+00 4.e+00 3.e+00]
 [1.e+00 4.e+00 5.e+00 4.e+00]]


We use the algorithm that performs elementary row operations:

In [21]:
def add_rows(A,k,i,j):
    "Adds k times the row j to row i in the matrix A"
    n=A.shape[0]
    E=np.eye(n)
    if i == j:
        E[i,i] = k+1
    else:
        E[i,j] = k
    return E@A

Let's eliminate the term below the pivot of the first row:

In [22]:
M1 = add_rows(M,-M[1,0]/M[0,0],1,0)
print(M1)

[[ 1.e-08  1.e+00  4.e+00  1.e+00]
 [ 0.e+00 -1.e+16 -4.e+16 -1.e+16]
 [ 1.e+00  4.e+00  5.e+00  4.e+00]]


Let's examine in detail the multiplier used in this elimination:

In [23]:
-M[1,0]/M[0,0]

-1e+16

This big number can bring problems when added to much smaller ones. For example:

In [24]:
-M[1,0]/M[0,0]+1.

-1e+16

Here the original number wasn't even changed after the addition. In the components that have been updated, the sum with the current component is practically irrelevant. In numerical computing when two numbers of different magnitudes are summed, the smaller number is converted to the base of the larger one, resulting in loss of accuracy.

Another potential problem of the simple Gaussian elimination procedure is that when the row pivot is zero the algorithm will be stopped by dividing by zero at the time of calculating the multiplier.

### 2. Algorithm for partial row pivoting

To avoid the problems described above we can proceed systematically during the Gaussian elimination process by always passing up the line with the highest pivot. This is the so-called Gaussian elimination algorithm with partial row pivoting.  Below we describe a pseudo-code that describes this algorithm:

| Steps | |
| --: | :-- |
|  1  | For $i = 1$, ..., $n$ do Steps 2-4
|  2  | $\phantom{--}$ Find $p$, where $p$ is the largest number with $i\leq p\leq n$
|  3  | $\phantom{--}$ If $p \neq i$, then exchange row $i$ with row $p$
|  4  | $\phantom{--}$ For $j=i+1, ..., n$ do Steps 5-6
|  5  | $\phantom{----}$ set $m_{ji} = a_{ji}/a_{ii}$
|  6  | $\phantom{----}$ Perform $E_j = ( E_j - m_{ji}E_i)$
|  7  | Set $x_n = a_{n,n+1}/a_{nn}$
|  8  | For $i = n-1, ..., 1$ do Step 9
|  9  | $$ x_i=\left.\left(a_{i,n+1}-\sum_{j=i+1}a_{ij}x_j \right) \middle/ a_{ii} \right.$$

The following function performs partial row pivoting:

In [25]:
import numpy as np

def partial_pivot(A, b):
    """
    Solve a linear system Ax = b using partial row pivoting.

    Parameters:
    A: numpy array of shape (n, n)
        The coefficient matrix.
    b: numpy array of shape (n,)
        The right-hand side vector.

    Returns:
    x: numpy array of shape (n,)
        The solution to the linear system Ax = b.
    """
    n = A.shape[0]
    for i in range(n):
        # Find the row with the largest absolute value in the ith column
        max_row = i
        for j in range(i+1, n):
            if abs(A[j,i]) > abs(A[max_row,i]):
                max_row = j

        # Swap rows i and max_row in A and b
        A[[i, max_row], :] = A[[max_row, i], :]
        b[[i, max_row]] = b[[max_row, i]]

        # Eliminate the ith variable from the remaining rows
        for j in range(i+1, n):
            factor = A[j,i] / A[i,i]
            A[j,i:] -= factor * A[i,i:]
            b[j] -= factor * b[i]

    # Back-substitution
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i,i+1:], x[i+1:])) / A[i,i]

    return np.transpose([x])

Let us test the algorithm:

In [28]:
A = np.array([[1.e-8, 1.,4.],[1.e+8, 1.,4.],[1.,4.,5.]])
B = np.array([1.,3.,4.])
Bt = B.reshape(3,1)

In [29]:

X3 = np.array([partial_pivot(A,B)])
X3

array([[[2.00000000e-08],
        [9.99999993e-01],
        [1.81818168e-09]]])

In [30]:
A = np.array([[1.e-8, 1.,4.],[1.e+8, 1.,4.],[1.,4.,5.]])
B = np.array([1.,3.,4.])
Bt = B.reshape(3,1)

In [32]:
R3=A@X3-Bt
R3

array([[[-1.11022302e-16],
        [ 0.00000000e+00],
        [ 0.00000000e+00]]])

Partial row pivoting is implemented in the numpy solver:

In [33]:
A = np.array([[1.e-8, 1.,4.],[1.e+8, 1.,4.],[1.,4.,5.]])
B = np.array([[1.],[3.],[4.]])

In [34]:
X1 = np.linalg.solve(A,B)
print(X1)

[[2.00000000e-08]
 [9.99999993e-01]
 [1.81818176e-09]]


### 3. Exercises

**1.** Let be the linear system $AX = B$ with

$$
A = \begin{bmatrix}
3.45 \times 10^{-10} & -4.34 & 5.98 \\
5.34 \times 10^{6} & 5.23 & 2.96 \\
4.39 \times 10^{10} & 4.32 & 15.32
\end{bmatrix}\,,\,\,
B = \begin{bmatrix}
1.98 \\ -8.39\\ 9.97
\end{bmatrix}
$$

(i) Solve the system step by step, using the Gaussian elimination process with partial row pivoting and calculate the residue. As a tool use the function

In [ ]:
def add_rows(A,k,i,j):
    "Add k times row j to row i"
    n=A.shape[0]
    E=np.eye(n)
    if i == j:
        E[i,i] = k+1
    else:
        E[i,j] = k
    return E@A

<br>
(ii) Solve the system using the partial_pivot function defined previously.
<br>
(iii) Solve the system using the numpy solver.

**2.** Solve problem 1 now with

$$
A = \begin{bmatrix}
0 & -4.34 & 5.98 & -4.32 \\
 3.45 & 333.23 & 2.96& 3.45 \\
3.97 & 4304.32 & 15.32& 432.203\\
433.97 & 4304.32 & 15.32& 432.203
\end{bmatrix}\,,\,\,
B = \begin{bmatrix}
1.98 \\ -8.39\\ 9.97 \\ 302.34
\end{bmatrix}
$$

**3.** Define a random 1000 x 1000 linear system. Use %timeit to measure the time necessary to solve this system using both the naîve Gauss elimination function GaussSolve:

In [ ]:
def GaussSolve(a,b):
    n = len(a)
    x=np.zeros(n)
    for k in range(0,n-1):
        for i in range(k+1,n):
            lam = a[i,k]/a[k,k]
            a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
            b[i] = b[i] - lam*b[k]
    x[n-1]=b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (b[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
    return np.transpose([x])

and the partial_pivot defined in this notebook. What are your conclusions?